In [1]:
import serial as sr
import time as tm
import numpy as np
from core import connect
import threading as tr

In [2]:
busy = False 

In [3]:
# get open port info
portinfo = connect.getOpenPorts() 

# 'windows' or 'mac'
deviceType = 'windows'

# MUST set baudrate in pump "System Settings", and MUST match this rate:
baudratePUMP = 9600
baudrateINO  = 38400

if deviceType == 'mac':
    print('macDevice')
    # initiate Connection object with first open port
    conn = connect.Connection(port = '/dev/tty.usbserial-A10MLXX5', baudrate = baudratePUMP, x = 0, mode = 0,verbose = False)

    arduino = sr.Serial(port='/dev/tty.usbmodem2101', baudrate=baudrateINO, timeout=1)
    tm.sleep(2)  # Give some time for the connection to establish
elif deviceType == 'windows':
    print('windowsDevice')
    # initiate Connection object with first open port
    conn = connect.Connection(port = 'COM3', baudrate = baudratePUMP, x = 0, mode = 0,verbose = False)

    arduino = sr.Serial(port='COM7', baudrate=baudrateINO, timeout=1)
    tm.sleep(2)  # Give some time for the connection to establish

windowsDevice


In [4]:
def WDseq(v,r):
    print("withdrawing", v, "mL, at", r,"ml/min")

    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = -v              # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    conn.setDiameter(diameter)  
    conn.setVolume(volume)      
    conn.setRate(rate)          
    conn.setDelay(delay)       
    tm.sleep(1)

    # Start pump
    conn.startPump()

In [5]:
def INseq(v,r):
    print("infusing", v, "mL, at", r,"ml/min")
    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = v              # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    conn.setDiameter(diameter)  
    conn.setVolume(volume)      
    conn.setRate(rate)          
    conn.setDelay(delay)    
       
    tm.sleep(1)

    # Start pump
    conn.startPump()

In [6]:
def unClogSeq(v,r):
    print("begin unclogging")
    global busy
    global running
    
    pump_buffer_time = 2.5
    slowInFactor = 0.25
    flushVolume = 2
    # big infuse
    INseq(v,r)
    tm.sleep(v / r * 60 + pump_buffer_time)

    # cycle 1
    WDseq(flushVolume,r)
    tm.sleep(flushVolume / r * 60 + pump_buffer_time)
    INseq(flushVolume,r * slowInFactor)
    tm.sleep(flushVolume / r / slowInFactor * 60 + pump_buffer_time)

    # # cycle 2
    # WDseq(2.5,r)
    # tm.sleep(2.5 / r * 60 + pump_buffer_time)
    # INseq(2.5,r)
    # tm.sleep(2.5 / r * 60 + pump_buffer_time)

    # # cycle 3
    # WDseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
    # INseq(5,r)
    # tm.sleep(5 / r * 60 + pump_buffer_time)
   
    busy    = False
    running = False

In [7]:
def isClog(pressure, clogLim, vinfed):
    global busy
    global t_clog
    global start_time


In [8]:
def runExp(v,r):
    # Open Connection to pump
    conn.openConnection()

    conn.stopPump()

    # Setup parameters for basic run
    units    = 'mL/min'		 	# OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
    diameter = 26.8             # 28.6mm diameter - can be set in pump GUI
    volume   = -v               # withdraw 10 mL volume 
    rate     = r                # 5 mL/min flow rate
    runtime  = volume/rate      # this is calculated implictly by pump
    delay    = 0                # 1 is 60 second delay

    # Communicate parameters to pump
    conn.setUnits(units)
    conn.setDiameter(diameter)  
    conn.setVolume(volume)      
    conn.setRate(rate)          
    conn.setDelay(delay)       

    tm.sleep(1)
    # Start pump
    conn.startPump()

In [9]:
# def read_arduino():
#     try:
#         arduino.flushInput()
#         data = arduino.readline()#.decode('utf-8').strip()
#         if data and data.isdigit():
#             return int(data)
#         else:
#             print("here")
#             return None
#     except:
#         # Catch any exception and ignore it
#         print("this here")
#         return None

def read_arduino():
    try:
        arduino.flushInput()  # Clear any junk in the input buffer
        data = arduino.readline().decode('utf-8').strip()
        return int(data)
    except:
        return None


In [10]:
# Function to read from Arduino in a separate thread
def read_arduino_thread():
    global latest_value
    while True:
        value = read_arduino()  # Your function to read from Arduino
        if value is not None:
            latest_value = value
        tm.sleep(0.25)  # Sleep to match Arduino sampling rate

In [11]:
import threading
import time as tm
import numpy as np

global running
global t_clog
global start_time
global latest_value

running = False
busy = False

data_list = []
time_list = []
time_list_1 = [0]
pressureOfClog = 86 

t_clog = []
latest_value = None

exp_run_rate = 0.5 # ml/min
exp_vol      = 30 # ml
start_time   = tm.time()


# Start the Arduino reading thread
thread = threading.Thread(target=read_arduino_thread)
thread.daemon = True  # Daemon thread will automatically close when the main program exits
thread.start()

try:
    while True:
        value = latest_value
         # print("value is", value, " time is ", tm.time()-start_time)
        tm.sleep(0.1)
        
        if value is not None and value > 85:
            if not running and not busy:
                running = True
                runExp(v = exp_vol, r = exp_run_rate)
                start_time_clog = tm.time()
                print("running exp for volume", exp_vol, " with flow rate of", exp_run_rate)

            time_running = tm.time() - start_time
            P = value
            cL = pressureOfClog

            if P <= cL and not busy:
                t_clog = tm.time() - start_time_clog 
                vinfed = exp_run_rate / 60 * t_clog
                print("pressure fell to ", P, "which is below clog limit of" , cL)
                print("Time to clog is ", t_clog)
                busy = True
                unClogSeq(v=vinfed, r=50)

                time_list_1.append(t_clog)

            round_time = np.round(tm.time() - start_time, 0)
        
except KeyboardInterrupt:
    print("Exiting...")
    conn.stopPump()
    


print("Data collection complete.")


running exp for volume 30  with flow rate of 0.5
pressure fell to  86 which is below clog limit of 86
Time to clog is  400.1490287780762
begin unclogging
infusing 3.334575239817301 mL, at 50 ml/min
withdrawing 2 mL, at 50 ml/min
infusing 2 mL, at 12.5 ml/min
running exp for volume 30  with flow rate of 0.5
pressure fell to  86 which is below clog limit of 86
Time to clog is  358.00893545150757
begin unclogging
infusing 2.9834077954292297 mL, at 50 ml/min
withdrawing 2 mL, at 50 ml/min
infusing 2 mL, at 12.5 ml/min
running exp for volume 30  with flow rate of 0.5
pressure fell to  86 which is below clog limit of 86
Time to clog is  39.30705904960632
begin unclogging
infusing 0.32755882541338605 mL, at 50 ml/min
withdrawing 2 mL, at 50 ml/min
infusing 2 mL, at 12.5 ml/min
running exp for volume 30  with flow rate of 0.5
pressure fell to  86 which is below clog limit of 86
Time to clog is  71.47735047340393
begin unclogging
infusing 0.5956445872783661 mL, at 50 ml/min
withdrawing 2 mL, at

In [12]:
print(time_list_1)

[0, 400.1490287780762, 358.00893545150757, 39.30705904960632, 71.47735047340393, 399.3158292770386, 497.12339520454407, 400.1133620738983, 280.8951425552368, 291.24882459640503, 238.9714217185974, 229.64787888526917, 202.69609761238098, 384.9667992591858, 71.97828459739685, 586.4217238426208, 666.2991554737091, 339.3929042816162, 161.46335816383362, 195.3358702659607, 410.1432909965515, 454.93386721611023, 395.50796008110046, 318.07663464546204, 132.70893478393555, 745.8835213184357, 464.663117647171, 251.31826400756836, 182.35644435882568, 320.19712829589844, 169.80443263053894, 187.20482468605042, 322.19110131263733, 233.7419216632843, 413.7043123245239, 268.8375597000122, 151.20295548439026, 466.5792052745819, 210.30659246444702, 278.8782842159271, 195.14411044120789, 234.04154133796692, 421.72431445121765, 271.72705698013306, 268.3278579711914]


In [ ]:
global running
global t_clog
global start_time

running = False

data_list = []
time_list = []
pressureOfClog = 90 

t_clog = []

exp_run_rate = 5 # ml/min
exp_vol      = 30 # ml
start_time   = tm.time()

try:
    while True:
        value = read_arduino()
        print("value is", value, " time is ", tm.time()-start_time)
        tm.sleep(0.1)
        if value is not None and value > 85:
            if running is not True and not busy:
                running = True
                runExp(v = exp_vol, r = exp_run_rate)
                start_time_clog = tm.time()
                print("running exp for volume", exp_vol, " with flow rate of", exp_run_rate)

            time_running = tm.time()-start_time
            #time_list.append(time_running)
            #data_list.append(value)
            #pressmBar = (value - 100) * 2000/398
            

            # isClog(pressure = value, clogLim = pressureOfClog, vinfed= volume)
            P = value
            cL = pressureOfClog

            # print("pressure is ", P)
            # print("clog lim is ", cL)
            if P <= cL and not busy:
                t_clog = tm.time() - start_time_clog 
                vinfed = exp_run_rate / 60 * t_clog
                print("pressure fell to ", P, "which is below clog limit of" , cL)
                print("Time to clog is ", t_clog)
                busy = True
                unClogSeq(v=vinfed,r=50)

            # Print the latest value every few seconds (e.g., every 5 seconds)
            round_time = np.round(tm.time() - start_time,0)
            # if round_time % 3 == 0:  # 0.1s * 30 = 3 seconds
            #     print(f"Latest value (after {round_time} seconds): {value}")
            #     # print("press mbar is", pressmBar)
        
except KeyboardInterrupt:
    print("Exiting...")
    # Handle any cleanup if necessary here

print("Data collection complete.")


In [13]:
time_list_1

[0,
 400.1490287780762,
 358.00893545150757,
 39.30705904960632,
 71.47735047340393,
 399.3158292770386,
 497.12339520454407,
 400.1133620738983,
 280.8951425552368,
 291.24882459640503,
 238.9714217185974,
 229.64787888526917,
 202.69609761238098,
 384.9667992591858,
 71.97828459739685,
 586.4217238426208,
 666.2991554737091,
 339.3929042816162,
 161.46335816383362,
 195.3358702659607,
 410.1432909965515,
 454.93386721611023,
 395.50796008110046,
 318.07663464546204,
 132.70893478393555,
 745.8835213184357,
 464.663117647171,
 251.31826400756836,
 182.35644435882568,
 320.19712829589844,
 169.80443263053894,
 187.20482468605042,
 322.19110131263733,
 233.7419216632843,
 413.7043123245239,
 268.8375597000122,
 151.20295548439026,
 466.5792052745819,
 210.30659246444702,
 278.8782842159271,
 195.14411044120789,
 234.04154133796692,
 421.72431445121765,
 271.72705698013306,
 268.3278579711914]